# Задание: предсказать цену жилья

Данные - 79 переменных, описывающих (почти) все аспекты жилых домов в Эймсе, штат Айова
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

In [558]:
import pandas as pd

data = pd.read_csv(r'C:\Users\David Hilbert\JupyterNotebook Projects\Предсказываем цену на жилье\train.csv')
test_data = pd.read_csv(r'C:\Users\David Hilbert\JupyterNotebook Projects\Предсказываем цену на жилье\test.csv')

# Сразу удаляем поле с идентификатором
data.drop(columns='Id', inplace=True)
test_data.drop(columns='Id', inplace=True)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Смотрим размер датасета

In [560]:
data.shape

(1460, 80)

In [561]:
test_data.shape

(1459, 79)

Смотрим из каких полей состоят данные:

In [563]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [564]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [565]:
numerical_features = list(data.columns[(data.dtypes == 'float64') | (data.dtypes == 'int64')])
categorical_features = list(data.columns[data.dtypes == object])

In [566]:
numerical_features_td = list(test_data.columns[(test_data.dtypes == 'float64') | (test_data.dtypes == 'int64')])
categorical_features_td = list(test_data.columns[test_data.dtypes == object])

Посмотрим внимательно на категориальные признаки:

In [568]:
data[categorical_features].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


Посмотрим в каких вещественных признаках есть пропуски и сколько их

In [570]:
data[numerical_features].isnull().sum()

MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

Заполним пропуски нулями

In [572]:
data[numerical_features] = data[numerical_features].fillna(0)

In [573]:
test_data[numerical_features_td] = test_data[numerical_features_td].fillna(0)

In [574]:
data[categorical_features].isnull().sum()

MSZoning            0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType        872
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinType2       38
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu       690
GarageType         81
GarageFinish       81
GarageQual         81
GarageCond         81
PavedDrive          0
PoolQC           1453
Fence            1179
MiscFeature      1406
SaleType            0
SaleCondition       0
dtype: int64

Посчитаем итоговое суммарное количество уникальных значение закодированных категориальных признаков

In [576]:
number_of_categorical = sum([data[field].nunique(dropna=False) for field in categorical_features])
number_of_categorical

267

In [577]:
number_of_categorical = sum([test_data[field].nunique(dropna=False) for field in categorical_features_td])
number_of_categorical

255

Кодируем категориальные признаки с помощью one-hot-encoding:

In [591]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()
encoder.fit(data[categorical_features])
categories = []
for i, feature in enumerate(categorical_features):
    categories.extend([f'{feature}: {value}' for value in encoder.categories_[i]])
data.loc[:, categories] = encoder.transform(data[categorical_features]).toarray().astype(int)

categories

['MSZoning: C (all)',
 'MSZoning: FV',
 'MSZoning: RH',
 'MSZoning: RL',
 'MSZoning: RM',
 'Street: Grvl',
 'Street: Pave',
 'Alley: Grvl',
 'Alley: Pave',
 'Alley: nan',
 'LotShape: IR1',
 'LotShape: IR2',
 'LotShape: IR3',
 'LotShape: Reg',
 'LandContour: Bnk',
 'LandContour: HLS',
 'LandContour: Low',
 'LandContour: Lvl',
 'Utilities: AllPub',
 'Utilities: NoSeWa',
 'LotConfig: Corner',
 'LotConfig: CulDSac',
 'LotConfig: FR2',
 'LotConfig: FR3',
 'LotConfig: Inside',
 'LandSlope: Gtl',
 'LandSlope: Mod',
 'LandSlope: Sev',
 'Neighborhood: Blmngtn',
 'Neighborhood: Blueste',
 'Neighborhood: BrDale',
 'Neighborhood: BrkSide',
 'Neighborhood: ClearCr',
 'Neighborhood: CollgCr',
 'Neighborhood: Crawfor',
 'Neighborhood: Edwards',
 'Neighborhood: Gilbert',
 'Neighborhood: IDOTRR',
 'Neighborhood: MeadowV',
 'Neighborhood: Mitchel',
 'Neighborhood: NAmes',
 'Neighborhood: NPkVill',
 'Neighborhood: NWAmes',
 'Neighborhood: NoRidge',
 'Neighborhood: NridgHt',
 'Neighborhood: OldTown',
 'Ne

In [607]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()
encoder.fit(test_data[categorical_features_td])
categories_td = []
for i, feature in enumerate(categorical_features_td):
    categories_td.extend([f'{feature}: {value}' for value in encoder.categories_[i]])
test_data.loc[:, categories_td] = encoder.transform(test_data[categorical_features_td]).toarray().astype(int)

categories_td

['MSZoning: C (all)',
 'MSZoning: FV',
 'MSZoning: RH',
 'MSZoning: RL',
 'MSZoning: RM',
 'MSZoning: nan',
 'Street: Grvl',
 'Street: Pave',
 'Alley: Grvl',
 'Alley: Pave',
 'Alley: nan',
 'LotShape: IR1',
 'LotShape: IR2',
 'LotShape: IR3',
 'LotShape: Reg',
 'LandContour: Bnk',
 'LandContour: HLS',
 'LandContour: Low',
 'LandContour: Lvl',
 'Utilities: AllPub',
 'Utilities: nan',
 'LotConfig: Corner',
 'LotConfig: CulDSac',
 'LotConfig: FR2',
 'LotConfig: FR3',
 'LotConfig: Inside',
 'LandSlope: Gtl',
 'LandSlope: Mod',
 'LandSlope: Sev',
 'Neighborhood: Blmngtn',
 'Neighborhood: Blueste',
 'Neighborhood: BrDale',
 'Neighborhood: BrkSide',
 'Neighborhood: ClearCr',
 'Neighborhood: CollgCr',
 'Neighborhood: Crawfor',
 'Neighborhood: Edwards',
 'Neighborhood: Gilbert',
 'Neighborhood: IDOTRR',
 'Neighborhood: MeadowV',
 'Neighborhood: Mitchel',
 'Neighborhood: NAmes',
 'Neighborhood: NPkVill',
 'Neighborhood: NWAmes',
 'Neighborhood: NoRidge',
 'Neighborhood: NridgHt',
 'Neighborhood:

# Обучение модели

Делим данные на обучение и тест:

In [610]:
from sklearn.model_selection import train_test_split

features = numerical_features + categories
features.remove('SalePrice')

X = data[features]
y = data['SalePrice']

Обучаем случайный лес с большим количеством деревьев:

In [612]:
%%time
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(n_estimators=500, n_jobs=4, verbose=1, oob_score=True,
                              max_features=X_train.shape[1] // 3)
model.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    1.0s finished


CPU times: total: 4.67 s
Wall time: 1.46 s


RandomForestRegressor(max_features=101, n_estimators=500, n_jobs=4,
                      oob_score=True, verbose=1)

In [625]:
features = numerical_features_td + categories_td

In [627]:
predictions = model.predict(features)

C:\Program Files\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'MSSubClass'

Для удобства создаем функцию, которая будет возвращать метрики сразу на обучающих и тестовых данных:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


def get_metrics(model, digits=0):
    results = []
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    results.append(['Train', mae, mse, r2 * 100])
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append(['Test', mae, mse, r2 * 100])
    return np.round(pd.DataFrame(columns=['Group', 'MAE', 'MSE', 'R2'], data=results), digits)

Смотрим метрики

In [ ]:
get_metrics(model)

In [ ]:
print(f'OOB MAE: {np.round(mean_absolute_error(y_train, model.oob_prediction_))}')
print(f'OOB MSE: {np.round(mean_squared_error(y_train, model.oob_prediction_)):.02e}')

Посмотрим как меняется метрика MAE в зависимости от количества деревьев:

In [ ]:
%%time
pred_train = np.hstack([estimator.predict(X_train.values).reshape((-1, 1)) for estimator in model.estimators_])
pred_test = np.hstack([estimator.predict(X_test.values).reshape((-1, 1)) for estimator in model.estimators_])

In [ ]:
mae = {'Train': [], 'Test': []}
for i in range(1, pred_train.shape[1]):
    mae['Train'].append(mean_absolute_error(y_train, pred_train[:, :i].mean(axis=1)))
for i in range(1, pred_train.shape[1]):
    mae['Test'].append(mean_absolute_error(y_test, pred_test[:, :i].mean(axis=1)))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
plt.plot(range(1, pred_train.shape[1]), mae['Train'], label='Train')
plt.plot(range(1, pred_test.shape[1]), mae['Test'], label='Test')
plt.xticks(ticks=np.arange(0, pred_train.shape[1] + 1, 25))
plt.title('Зависимость MAE от количества деревьев в случайном лесе')
plt.xlabel('Number of trees in a forest')
plt.ylabel('MAE')
plt.legend()
plt.show()

Для примера возьмем 150 деревьев

In [ ]:
model = RandomForestRegressor(n_estimators=150, n_jobs=4, verbose=1, oob_score=True)
model.fit(X_train, y_train)

In [ ]:
get_metrics(model)

# Выведем признаки с наибольшей важностью

In [ ]:
pd.DataFrame(data={'feature': features, 
                   'importance': model.feature_importances_}).sort_values('importance', ascending=False).iloc[:10]